# 간단한 추천시스템 만들기

1. MovieLens 데이터셋 불러오기
2. MovieLens 데이터셋 중 학습셋과 평가셋 나누기
3. 간단한 추천 알고리즘 만들기(평점을 예측하고 평가는 RMSE로 한다)

# 필요한 라이브러리 정의(Configuration)

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix

import warnings
warnings.filterwarnings("ignore")

#MovieLens 데이터셋 불러오기

- ratings.csv, movies.csv, tags.csv 불러오기

In [10]:
# Google drive는 경로 변경
path = '/content/drive/MyDrive/data/movielens/'

In [11]:
ratings_df = pd.read_csv(os.path.join(path, 'ratings.csv'), encoding='utf-8')
movies_df = pd.read_csv(os.path.join(path, 'movies.csv'), index_col='movieId', encoding='utf-8')
tags_df = pd.read_csv(os.path.join(path, 'tags.csv'), encoding='utf-8')

In [ ]:
print(ratings_df.shape)
print(ratings_df.head())

# ratings 데이터 정보 확인하기

* 몇 명의 유저가 몇 개의 영화에 평점을 줬는지
* 각 유저가 어떤 영화에 평점을 줬는지 sparse matrix

In [12]:
num_users = ratings_df['userId'].unique()
num_movies = ratings_df['movieId'].unique()

print("총 유저 수: ", len(num_users))
print("총 영화 수: ", len(num_movies))

총 유저 수:  610
총 영화 수:  9724


In [13]:
# pivot ratings into movie features
user_movie_matrix = ratings_df.pivot(
    index='movieId',
    columns='userId',
    values='rating'
).fillna(0)
 
# convert dataframe of movie features to scipy sparse matrix
sparse_mat = csr_matrix(user_movie_matrix.values)

In [14]:
print(user_movie_matrix)

userId   1    2    3    4    5    6    7    ...  604  605  606  607  608  609  610
movieId                                     ...                                   
1        4.0  0.0  0.0  0.0  4.0  0.0  4.5  ...  3.0  4.0  2.5  4.0  2.5  3.0  5.0
2        0.0  0.0  0.0  0.0  0.0  4.0  0.0  ...  5.0  3.5  0.0  0.0  2.0  0.0  0.0
3        4.0  0.0  0.0  0.0  0.0  5.0  0.0  ...  0.0  0.0  0.0  0.0  2.0  0.0  0.0
4        0.0  0.0  0.0  0.0  0.0  3.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
5        0.0  0.0  0.0  0.0  0.0  5.0  0.0  ...  3.0  0.0  0.0  0.0  0.0  0.0  0.0
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
193581   0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
193583   0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
193585   0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
193587   0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1936

In [26]:
print(list(user_movie_matrix[1].value_counts())[1:])

[124, 76, 26, 5, 1]


In [15]:
print(sparse_mat)

  (0, 0)	4.0
  (0, 4)	4.0
  (0, 6)	4.5
  (0, 14)	2.5
  (0, 16)	4.5
  (0, 17)	3.5
  (0, 18)	4.0
  (0, 20)	3.5
  (0, 26)	3.0
  (0, 30)	5.0
  (0, 31)	3.0
  (0, 32)	3.0
  (0, 39)	5.0
  (0, 42)	5.0
  (0, 43)	3.0
  (0, 44)	4.0
  (0, 45)	5.0
  (0, 49)	3.0
  (0, 53)	3.0
  (0, 56)	5.0
  (0, 62)	5.0
  (0, 63)	4.0
  (0, 65)	4.0
  (0, 67)	2.5
  (0, 70)	5.0
  :	:
  (9700, 337)	2.5
  (9701, 337)	3.0
  (9702, 183)	4.0
  (9702, 247)	3.5
  (9703, 317)	2.5
  (9704, 209)	1.0
  (9705, 461)	2.5
  (9706, 49)	3.5
  (9707, 337)	1.5
  (9708, 337)	4.0
  (9709, 337)	1.0
  (9710, 337)	1.5
  (9711, 337)	1.0
  (9712, 337)	1.0
  (9713, 183)	4.5
  (9714, 183)	3.5
  (9715, 183)	3.0
  (9716, 183)	4.0
  (9717, 183)	4.0
  (9718, 183)	3.5
  (9719, 183)	4.0
  (9720, 183)	3.5
  (9721, 183)	3.5
  (9722, 183)	3.5
  (9723, 330)	4.0


In [27]:
# 사용자별로 평가한 영화 갯수 정리 (0점인 데이터는 제외) 
user_info_df = pd.DataFrame(data = [sum(list(user_movie_matrix[int(x)].value_counts())[1:]) for x in user_movie_matrix.columns],
                           index = user_movie_matrix.columns, columns=['movies_rated'])


In [28]:
user_info_df

,movies_rated
userId,
1,232
2,29
3,39
4,216
5,44
...,...
606,1115
607,187
608,831


In [31]:
# 영화별로 평가한 사용자 수 정리 (0점인 데이터는 제외) 
movie_info_df = pd.DataFrame(data = [sum(list(user_movie_matrix.loc[int(x)].value_counts())[1:]) for x in user_movie_matrix.index],
                           index = user_movie_matrix.index, columns=['users_rated'])

In [32]:
movie_info_df

,users_rated
movieId,
1,215
2,110
3,52
4,7
5,49
...,...
193581,1
193583,1
193585,1


# MovieLens 데이터셋 중 학습셋과 평가셋 나누기

In [34]:
train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state=1234)

In [35]:
print(train_df.shape)
print(test_df.shape)

(80668, 4)
(20168, 4)


# test set에는 존재하지만, train set에는 없는 영화 또는 사용자 비율 파악

In [36]:
# 집합 A - 집합 B => 집합 B에는 없고 집합 A에만 있는 item 

# userId
print("사용자: ",len(list(set(test_df['userId'].unique()) - set(train_df['userId'].unique()))))

# movieId
print("영화: ", len(list(set(test_df['movieId'].unique()) - set(train_df['movieId'].unique()))))
print("test set의 전체 영화 수: ",  len(test_df['movieId'].unique()))

사용자:  0
영화:  786
test set의 전체 영화 수:  5171


In [37]:
movies_not_included = list(set(test_df['movieId'].unique()) - set(train_df['movieId'].unique()))
print(sorted(movies_not_included)[:10])

not_included_df = test_df[test_df.movieId.isin(movies_not_included)].sort_values(by='movieId')
print(not_included_df.head(10))

print("train set에 없고, test set에만 있는 영화 데이터 수: ", not_included_df.shape)

[49, 117, 137, 178, 241, 320, 359, 478, 488, 495]
       userId  movieId  rating   timestamp
29386     202       49     3.0   974925453
97066     604      117     3.0   832080636
99501     609      137     3.0   847221054
27959     191      178     1.0   829760898
98493     607      241     4.0   964744490
96182     603      320     3.0   953925390
728         6      359     3.0   845556412
92825     599      478     2.5  1498515125
73214     474      488     3.0  1047569232
96218     603      495     5.0   953927108
train set에 없고, test set에만 있는 영화 데이터 수:  (852, 4)


# 간단한 추천알고리즘 만들기

1. 랜덤으로 평점 예측하기
2. 영화 평균 평점기반 예측하기
3. 사용자 평균 평점기반 예측하기
4. Rule기반 영화 랭킹 예측하기

## 랜덤으로 평점 예측하기

In [38]:
# 0.5 - 5.0사이의 숫자를 예측해야할 평점 수 만큼 generate
ratings_range = np.arange(0.5, 5.5, step=0.5)
ratings_range

array([0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ])

In [39]:
import random
pred_random = [random.choice(ratings_range) for x in range(len(test_df))]
pred_random[:10]

[5.0, 5.0, 2.5, 1.0, 4.5, 2.5, 5.0, 4.0, 1.5, 0.5]

In [40]:
test_df['pred_ratings_random'] = pred_random

In [41]:
mse = mean_squared_error(y_true=test_df['rating'].values, y_pred=test_df['pred_ratings_random'].values)
rmse = np.sqrt(mse)

print(mse, rmse)

3.7325094208647362 1.9319703467871179


## 영화 평균 평점기반 예측하기

1. train set의 모든 영화에 대해서 평균 평점 구하기
2. test set 예측할 때, train set의 영화 평균 평점 활용하기. 만약 없다면, random으로 선택하기.

In [42]:
train_movie_df = train_df.groupby('movieId').mean()

print(train_movie_df.shape)
print(train_movie_df.head())

(8938, 3)
             userId    rating     timestamp
movieId                                    
1        307.473373  3.893491  1.128439e+09
2        327.475610  3.396341  1.142893e+09
3        266.386364  3.454545  9.900434e+08
4        192.750000  2.250000  8.425133e+08
5        309.526316  3.039474  1.007415e+09


In [46]:
def avg_rating_prediction(training_set, x):
    if x in training_set.index:
        pred_rating = training_set.loc[x]['rating']
    else:
        pred_rating = random.choice(ratings_range)
    return pred_rating

In [47]:
test_df['pred_rating_movie'] = test_df['movieId'].apply(lambda x: avg_rating_prediction(train_movie_df, x))

test_df.head()

,userId,movieId,rating,timestamp,pred_ratings_random,pred_rating_movie
99731,610,3527,5.0,1479545223,5.0,3.604167
97583,606,1250,3.5,1171376891,5.0,4.180556
38197,262,213,5.0,840310907,2.5,3.750000
11474,68,69406,3.0,1261622505,1.0,3.571429
34105,232,4728,3.0,1218166950,4.5,2.769231


In [48]:
mse = mean_squared_error(y_true=test_df['rating'].values, y_pred=test_df['pred_rating_movie'].values)
rmse = np.sqrt(mse)

print(mse, rmse)

1.052417410919426 1.0258739741895326


## 사용자 평균 평점기반 예측하기

1. train set의 모든 유저가 준 평균 평점
2. test set 예측할 때, 유저가 train set에서 준 평균 평점을 활용. 유저가 없을 경우 random 평점 적용

In [49]:
train_user_df = train_df.groupby('userId').mean()

print(train_user_df.shape)
print(train_user_df.head())

(610, 3)
             movieId    rating     timestamp
userId                                      
1        1891.168478  4.320652  9.649865e+08
2       70402.760000  3.940000  1.445715e+09
3        8394.733333  2.516667  1.306464e+09
4        1957.923077  3.631868  9.655941e+08
5         337.606061  3.636364  8.474351e+08


In [50]:
test_df['pred_rating_user'] = test_df['userId'].apply(lambda x: avg_rating_prediction(train_user_df, x))

test_df.head()

,userId,movieId,rating,timestamp,pred_ratings_random,pred_rating_movie,pred_rating_user
99731,610,3527,5.0,1479545223,5.0,3.604167,3.678709
97583,606,1250,3.5,1171376891,5.0,4.180556,3.649718
38197,262,213,5.0,840310907,2.5,3.750000,2.925000
11474,68,69406,3.0,1261622505,1.0,3.571429,3.229331
34105,232,4728,3.0,1218166950,4.5,2.769231,3.242268


In [51]:
mse = mean_squared_error(y_true=test_df['rating'].values, y_pred=test_df['pred_rating_user'].values)
rmse = np.sqrt(mse)

print(mse, rmse)

0.8905889036428333 0.9437101798978504


## Rule 기반 영화 평점 예측하기

1. train set에 포함된 유저의 영화 평균 평점과 영화의 장르를 활용하여, 장르별 평균 평점 계산 -> test set의 영화 장르의 평균 평점으로 예측